# MiniCascade-RAG Agent Graph 可视化

这个notebook用于可视化LangGraph agent的连接关系和工作流程。

## Agent架构说明

根据代码分析，您的agent架构包含以下组件：

1. **Intent Detection** - 意图识别
2. **Entity Extraction** - 实体提取  
3. **Context Resolution** - 上下文解析
4. **Query Rewrite** - 查询改写
5. **RAG Retrieval** - RAG检索
6. **Chat** - 对话生成
7. **Tool Call** - 工具调用

## 工作流程

```
Start → Intent Detection → Route by Intent
                           ↓
                     [qa/write path]
                           ↓
Entity Extraction → Context Resolution → Query Rewrite → RAG Retrieval → Chat ⇄ Tool Call → End
                           ↓
                     [other paths] 
                           ↓
                         Chat ⇄ Tool Call → End
```


In [ ]:
# 导入必要的模块
import sys
from pathlib import Path

# 添加项目路径
project_root = Path.cwd()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"项目根目录: {project_root}")
print(f"Python路径: {sys.path[:3]}...")

In [ ]:
# 导入项目模块
try:
    from app.core.agent.graph.chief_agent import LangGraphAgent

    print("✅ 成功导入项目模块")
except ImportError as e:
    print(f"❌ 导入失败: {e}")
    print("请确保：")
    print("1. 在项目根目录运行此notebook")
    print("2. 已安装所有依赖")
    print("3. 配置文件正确")

In [ ]:
# 创建和可视化agent图
async def create_and_visualize_graph():
    """创建并可视化agent图"""
    try:
        print("🚀 创建LangGraph Agent...")
        agent = LangGraphAgent()

        print("🔨 构建图结构...")
        graph = await agent.create_graph()

        if graph is None:
            print("❌ 图创建失败")
            return None

        print("✅ 图创建成功!")
        return graph

    except Exception as e:
        print(f"❌ 创建图时出错: {e}")
        import traceback

        traceback.print_exc()
        return None


# 运行异步函数
graph = await create_and_visualize_graph()

In [ ]:
# 显示图的基本信息
if graph:
    graph_info = graph.get_graph()

    print("📊 图结构信息:")
    print(f"  节点数量: {len(graph_info.nodes)}")
    print(f"  边数量: {len(graph_info.edges)}")
    print(f"  入口点: {graph_info.first_node}")

    print("\n🔍 所有节点:")
    for i, node_id in enumerate(graph_info.nodes, 1):
        print(f"  {i:2d}. {node_id}")

    print("\n🔗 边连接关系:")
    for i, edge in enumerate(graph_info.edges, 1):
        print(f"  {i:2d}. {edge.source} → {edge.target}")

In [ ]:
# Mermaid图定义和可视化
if graph:
    try:
        print("🎨 Mermaid图定义:")
        print("=" * 50)
        mermaid_def = graph.get_graph().draw_mermaid()
        print(mermaid_def)
        print("=" * 50)

        # 保存到文件
        with open("agent_graph.mermaid", "w", encoding="utf-8") as f:
            f.write(mermaid_def)
        print("\n💾 Mermaid图已保存到 agent_graph.mermaid")
        print("💡 将上面的代码复制到 https://mermaid.live 查看交互式图形")

        # 尝试生成PNG图片并显示
        try:
            print("\n🖼️ 生成PNG图片...")
            png_data = graph.get_graph().draw_mermaid_png()

            with open("agent_graph.png", "wb") as f:
                f.write(png_data)
            print("✅ PNG图片已保存到 agent_graph.png")

            # 在Jupyter中显示图片
            try:
                from IPython.display import Image, display

                display(Image(png_data))
            except ImportError:
                print("⚠️ 无法在当前环境显示图片，请查看生成的PNG文件")

        except Exception as e:
            print(f"PNG生成失败: {e}")
            print("💡 提示: 需要安装 playwright 和 kaleido 才能生成PNG")
            print("   pip install playwright kaleido")
            print("   playwright install")

    except Exception as e:
        print(f"Mermaid可视化失败: {e}")